In [11]:
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Conv2D, ReLU, Flatten, Dense, Softmax, BatchNormalization
from tensorflow.keras.layers import Input, Activation, Add, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, Nadam, SGD
# from tensorflow.keras.applications.resnet50 import ResNet50
from queue import Queue
import keras
import numpy as np

In [12]:
tf.__version__
tf.test.is_gpu_available()

True

# Data Pre-Processing

Open **kyu_train.csv** file and split the games into a list.
Every row of csv: `KL0000000001,B,B[pq],W[dd],B[dp],W[pd],B[jc],...`. 

Columns are:

    1. KL0000000001: Game ID
    2. B: Player's color
    3-... : Moves
    
We cropped only the moves to game list as:

In [13]:
df = open('./Training Dataset/kyu_train.csv').read().splitlines()
games = [i.split(',',2)[-1] for i in df]
colors = [i.split(',',2)[1] for i in df]

Create a dictionary to convert the coordinates from characters to numbers

In [14]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
chartonumbers = {k:v for k,v in enumerate(chars)}
coordinates

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18}

We decided to build a DCNN model in this tutorial. We create data samples by using every move in every game, meaning that the target is to predict the next move by feeding the previous state of the table in every game for every move. Therefore, we can collect much more data samples from games.

For the simplicity, we used 4 dimensional feature map to represent the data as below:
 1. Positions of black stones: mark them as 1 and the rest of the table as 0
 2. Positions of white stones: mark them as 1 and the rest of the table as 0
 3. Empty areas of the table: mark the empty areas as 1 and occupied areas as 0
 4. The last move in the table: mark the position of the last move as 1 and the rest as 0
 
Target value is a number between 0-361(19\*19). Later this will be one-hot encoded.

In [15]:
liber = Queue()
def count_air2(move):
    bfs = []
    col = coordinates[move[2]]
    row = coordinates[move[3]]
    #找四周要bfs的
    bfs.append([row, col])
    if col - 1 >= 0 and x[row][col - 1][8] == 0:
        bfs.append([row, col - 1])
    if col + 1 < 19 and x[row][col + 1][8] == 0:
        bfs.append([row, col + 1])
    if row - 1 >= 0 and x[row - 1][col][8] == 0:
        bfs.append([row - 1, col])
    if row + 1 < 19 and x[row + 1][col][8] == 0:
        bfs.append([row + 1, col])
    for m in bfs:
        BFS(m)
    bfs = []
    if not liber.empty():
#         print("liber")
        while not liber.empty():
            li_row, li_col = liber.get()
#             x[li_row][li_col][8] = 1
            if li_col - 1 >= 0 and x[li_row][li_col - 1][8] == 0:
                bfs.append([li_row, li_col - 1])
            if li_col + 1 < 19 and x[li_row][li_col + 1][8] == 0:
                bfs.append([li_row, li_col + 1])
            if li_row - 1 >= 0 and x[li_row - 1][li_col][8] == 0:
                bfs.append([li_row - 1, li_col])
            if li_row + 1 < 19 and x[li_row + 1][li_col][8] == 0:
                bfs.append([li_row + 1, li_col])
        for m in bfs:
            BFS(m)
        
def BFS(now):
    q = Queue()
    row = now[0]
    col = now[1]
    q.put((row, col))
    visited = [[False for _ in range(19)] for _ in range(19)]
    nodes = []
    air = 0
    next_board = 0
    board = player
    if oppnent[row, col] == 1:
        board = oppnent
        next_board = 4
    while not q.empty():
        m, n = q.get()
        if m < 0 or n < 0 or m >= 19 or n >= 19 or visited[m][n]:
            continue
        visited[m][n] = True
        if board[m, n] == 1:
            nodes.append([m, n])
            q.put((m - 1, n))
            q.put((m + 1, n))
            q.put((m, n - 1))
            q.put((m, n + 1))
        elif x[m, n, 8] == 1: #空地 -> 自由度+1
            air += 1
    dele = -1
    for i in range(8):
        if x[row][col][i] == 1:
            dele = i
            break
    air = min(air, 4)
    for node in nodes:
        n_row = node[0]
        n_col = node[1]
        #原本的改0，現在的氣數對應的棋盤改1
        if dele != -1:
            x[n_row][n_col][dele] = 0;
        if air == 0:
            x[n_row][n_col][8] = 1
            x[n_row, n_col, 17] = 0
            x[n_row, n_col, 18] = 0
            player[n_row, n_col] = 0
            oppnent[n_row, n_col] = 0
            liber.put(node)
        else:
            x[n_row][n_col][next_board + air - 1] = 1

In [16]:
#0~3:所有跟最後一步顏色相同的氣(1, 2, 3, >4)
#4~7:所有跟最後一步顏色不同的氣(1, 2, 3, >4)
#8:標示空地
#9~16:最後8步
#17:周圍顏色相同的7*7
#18:周圍顏色不同的7*7

x = np.zeros((19,19,19))
player = np.zeros((19, 19))
oppnent = np.zeros((19, 19))


def prepare_input(moves, player_color):
#     player_color = moves[-1][0] #
    sz = len(moves)
    if sz == 0:
        return x;
    move = moves[-1]
    
    color = move[0]
    column = coordinates[move[2]]
    row = coordinates[move[3]]
    if color == player_color: #
        player[row,column] = 1
    else: #
        oppnent[row,column] = 1
    x[row,column,8] = 0
    
    #倒數8步
    for i in range(16, 9, -1):
        x[:, :, i] = x[:, :, i - 1]
#     x[:, :, 10:16] = np.copy(x[:, :, 9:15])
    x[:, :, 9] = 0
#     x[:, :, 9] = np.zeros((19, 19))
    x[row, column, 9] = 1
#     for i in range(8):
#         if sz - 1 - i - 1 >= 0:
#             col_l = coordinates[moves[sz - 1 - i - 1][2]]
#             row_l = coordinates[moves[sz - 1 - i - 1][3]]
#             x[row_l, col_l, 9 + i] = 0
#             if i != 7:
#                 x[row_l, col_l, 9 + i + 1] = 1
#     x[row, column, 9] = 1
    #周圍7*7
    x[:, :, 17] = 0
#     x[:, :, 17] = np.zeros((19, 19))
    x[:, :, 18] = 0
#     x[:, :, 18] = np.zeros((19, 19))
    last_col = coordinates[move[2]]
    last_row = coordinates[move[3]]
    rad = 3 #要改範圍大小的話改這個
    row1 = max(0, last_row - rad)
    row7 = min(18, last_row + rad)
    col1 = max(0, last_col - rad)
    col7 = min(18, last_col + rad)
    for i in range(row1, row7 + 1, 1):
        for j in range(col1, col7 + 1, 1):
            x[i, j, 17] = player[i, j]
            x[i, j, 18] = oppnent[i, j]
    
    count_air2(move)
#     for i in range(8, 9):
#         print("Case ", end = '')
#         print(i)
#         for j in range(0, 19):
#             for k in range(0, 19):
#                 if x[j][k][i] == 1:
#                     print(chartonumbers[k], chartonumbers[j])
#                 print(int(x[j][k][i]), end = ' ')
#             print()
#         print()
    
    #列印所有棋盤
#     for i in range(0, 3, 1):
#         print("  a b c d e f g h i j k l m n o p q r s")
#         for j in range(0, 19, 1):
#             print(chars[j], end = " ")
#             for k in range(0, 19, 1):
#                 print(int(x[j, k, i]), end = " ")
#             print("")
#         print("")
    
    return x

def prepare_label(move):
    column = coordinates[move[2]]
    row = coordinates[move[3]]
    return column*19+row

In [17]:
def data_generator(games, batch_size, indexs):
    def generator():
        x_batch = [] # Initialize data batch
        y_batch = [] # Initialize target batch
        for game_i in indexs: # Iterate through games
            x = np.zeros((19,19,19))
            moves_list = games[game_i].split(',')
#             print(moves_list)
            indexs_ = np.arange(len(moves_list))
            np.random.shuffle(indexs_)
        
            for count in indexs_:
                if colors[game_i] == moves_list[count][0]:
#                     print(move)
                    x_batch.append(prepare_input(moves_list[:count], colors[game_i]))
#                     print(count)
                    y_batch.append(prepare_label(moves_list[count]))
                    if len(x_batch) == batch_size: # Yield when reached batch size
                        yield np.array(x_batch), (tf.one_hot(np.array(y_batch), depth=19*19)).numpy()
                        x_batch = []
                        y_batch = []
    return generator
#         for game_i, game in enumerate(games): # Iterate through games
#             x = np.zeros((19,19,13))
#             moves_list = game.split(',')
# #             print(moves_list)
#             for count, move in enumerate(moves_list):
#                 if colors[game_i] == move[0]:
# #                     print(move)
#                     x_batch.append(prepare_input(moves_list[:count], colors[game_i]))
#                     y_batch.append(prepare_label(moves_list[count]))
#                     if len(x_batch) == batch_size: # Yield when reached batch size
#                         yield np.array(x_batch), (tf.one_hot(np.array(y_batch), depth=19*19)).numpy()
#                         x_batch = []
#                         y_batch = []
#     return generator

# batch_size = 64
# val_rate = 0.3 # 0.1 means 0.1 for val ,0.9 for training
# split_point = int(len(games) * (1 - val_rate))
# generator = data_generator(games[:split_point], batch_size)
# dataset = tf.data.Dataset.from_generator(generator, 
#                                          output_types=(tf.float32, tf.float32),
#                                          output_shapes=(tf.TensorShape((batch_size,19,19,13)),tf.TensorShape((batch_size,361)))
#                                         )
# # SHUFFLE_BUFFER_SIZE = 200
# # dataset = dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(batch_size)
# dataset = dataset.prefetch(tf.data.AUTOTUNE)

In [18]:
# Check how many samples can be obtained
n_games = 0
n_moves = 0
for game in games:
    n_games += 1
    moves_list = game.split(',')
    for move in moves_list:
        n_moves += 1
print(f"Total Games: {n_games}, Total Moves: {n_moves}")

Total Games: 118500, Total Moves: 27135638


In [19]:
batch_size = 64
val_start = 0.8
val_stop = 1.0

indexs = np.arange(int(n_games * val_stop))
np.random.shuffle(indexs)

t_games = 0
t_moves = 0
for i in indexs[:int(len(indexs) * val_start)]:
    t_games += 1
    moves_list = games[i].split(',')
    for move in moves_list:
        if colors[i] == move[0]:
            t_moves += 1

print(f"Train Games: {t_games}, Train Moves: {t_moves}")
v_games = 0
v_moves = 0

for i in indexs[int(len(indexs) * val_start):int(len(indexs) * val_stop)]:
    v_games += 1
    moves_list = games[i].split(',')
    for move in moves_list:
        if colors[i] == move[0]:
            v_moves += 1
print(f"Val Games: {v_games}, Val Moves: {v_moves}")

train_steps = np.ceil((t_moves / batch_size) - 1)
val_steps = np.ceil((v_moves / batch_size) - 1)
print(f"train_steps: {train_steps}")
print(f"val_steps: {val_steps}")
data_gen = data_generator(games, batch_size, indexs[:int(len(indexs) * val_start)]) # 90% of the complete dataset
dataset = tf.data.Dataset.from_generator(data_gen, 
                                         output_types=(tf.dtypes.float32, tf.dtypes.float32),
                                         output_shapes=(tf.TensorShape((batch_size,19,19,19)),tf.TensorShape((batch_size,361))))
# dataset = dataset.batch(batch_size, drop_remainder=True)
dataset = dataset.prefetch(tf.data.AUTOTUNE)

data_gen_valid = data_generator(games, batch_size
                                , indexs[int(len(indexs) * val_start):int(len(indexs) * val_stop)])
dataset_valid = tf.data.Dataset.from_generator(data_gen_valid, 
                                         output_types=(tf.dtypes.float32, tf.dtypes.float32),
                                         output_shapes=(tf.TensorShape((batch_size,19,19,19)),tf.TensorShape((batch_size,361))))
# dataset_valid = dataset_valid.batch(batch_size, drop_remainder=True)
dataset_valid = dataset_valid.prefetch(tf.data.AUTOTUNE)

Train Games: 94800, Train Moves: 10852633
Val Games: 23700, Val Moves: 2714729
train_steps: 169572.0
val_steps: 42417.0


In [20]:
# batch_size = 64
# train_start = 0.0
# val_start = 0.5
# val_stop = 1.0

# dataset_size = len(games)
# t_games = 0
# t_moves = 0
# tmp = int(dataset_size * train_start)
# for game in games[int(dataset_size * train_start):int(dataset_size * val_start)]:
#     t_games += 1
#     moves_list = game.split(',')
#     for move in moves_list:
#         if colors[tmp] == move[0]:
#             t_moves += 1
#     tmp += 1
# print(f"Train Games: {t_games}, Train Moves: {t_moves}")
# v_games = 0
# v_moves = 0
# tmp = int(dataset_size * val_start)
# for game in games[int(dataset_size * val_start):int(dataset_size * val_stop)]:
#     v_games += 1
#     moves_list = game.split(',')
#     for move in moves_list:
#         if colors[tmp] == move[0]:
#             v_moves += 1
#     tmp += 1
# print(f"Val Games: {v_games}, Val Moves: {v_moves}")

# train_steps = int(t_moves /batch_size)
# val_steps = int(v_moves /batch_size)

# data_gen = data_generator(games[int(dataset_size * train_start):int(dataset_size * val_start)], batch_size) # 90% of the complete dataset
# dataset = tf.data.Dataset.from_generator(data_gen, 
#                                          output_types=(tf.dtypes.float32, tf.dtypes.float32),
#                                          output_shapes=(tf.TensorShape((batch_size,19,19,13)),tf.TensorShape((batch_size,361))))
# # dataset = dataset.batch(batch_size, drop_remainder=True)
# dataset = dataset.prefetch(tf.data.AUTOTUNE)

# data_gen_valid = data_generator(games[int(dataset_size * val_start):int(dataset_size * (val_stop))], 
#                                 batch_size,
#                                 shuffle = False)
# dataset_valid = tf.data.Dataset.from_generator(data_gen_valid, 
#                                          output_types=(tf.dtypes.float32, tf.dtypes.float32),
#                                          output_shapes=(tf.TensorShape((batch_size,19,19,13)),tf.TensorShape((batch_size,361))))
# # dataset_valid = dataset_valid.batch(batch_size, drop_remainder=True)
# dataset_valid = dataset_valid.prefetch(tf.data.AUTOTUNE)

The code below is run for baseline model only by using only the first 500 games from the dataset. You might need to create a data generator to use complete dataset. Otherwise your RAM might not enough to store all (If you run the code on free version of Google Colab, it will crash above 500 game samples).

# Training

### Simple DCNN Model:

In [25]:
def residual_block(x, filters, kernel_size):
    y = Conv2D(kernel_size=kernel_size,
               filters=filters,
               padding='same')(x)
    y = ReLU()(y)
    y = Conv2D(kernel_size=kernel_size,
               filters=filters,
               padding='same')(y)
    output = Add()([x,y])
    output = ReLU()(output)
    return output

def residual_block_s(x, filters, kernel_size):
    y = Conv2D(kernel_size=kernel_size,
               filters=filters,
               padding='same')(x)
    y = ReLU()(y)
    y = Conv2D(kernel_size=kernel_size,
               filters=filters,
               padding='same')(y)
    x = Conv2D(kernel_size = 1,
              filters=filters)(x)
    output = Add()([x,y])
    output = ReLU()(output)
    return output

def go_res():
    inputs = Input(shape=(19, 19, 19))
    conv5x5 = Conv2D(kernel_size=5,
                     filters=64,
                     padding="same",
                     name='conv5x5')(inputs)
    conv1x1 = Conv2D(kernel_size=1,
                     filters=64,
                     padding="same",
                     name='conv1x1')(inputs)
    outputs = Add()([conv5x5, conv1x1])
    outputs = ReLU()(outputs)
    outputs = residual_block(x=outputs,
                             filters=64,
                             kernel_size=3)
    outputs = residual_block(x=outputs,
                             filters=64,
                             kernel_size=3)
    outputs = residual_block(x=outputs,
                             filters=64,
                             kernel_size=3)
    outputs = residual_block(x=outputs,#
                             filters=64,
                             kernel_size=3)
    outputs = residual_block(x=outputs,
                             filters=64,
                             kernel_size=3)
    outputs = residual_block(x=outputs,
                             filters=64,
                             kernel_size=3)
    outputs = residual_block(x=outputs,
                             filters=64,
                             kernel_size=3)
    outputs = residual_block(x=outputs,
                             filters=64,
                             kernel_size=3)
    outputs = residual_block(x=outputs,
                             filters=64,
                             kernel_size=3)
    outputs = residual_block(x=outputs,
                             filters=64,
                             kernel_size=3)
    outputs = residual_block(x=outputs,
                             filters=64,
                             kernel_size=3)
    outputs = residual_block(x=outputs,
                             filters=64,
                             kernel_size=3)
    outputs = residual_block(x=outputs,
                             filters=64,
                             kernel_size=3)
    outputs = residual_block(x=outputs,#
                             filters=64,
                             kernel_size=3)
    outputs = residual_block(x=outputs,
                             filters=64,
                             kernel_size=3)
    outputs = residual_block(x=outputs,
                             filters=64,
                             kernel_size=3)
    outputs = residual_block(x=outputs,
                             filters=64,
                             kernel_size=3)
#     outputs = Conv2D(kernel_size=3,
#                      filters=1,
#                      padding="same")(outputs)
#     outputs = ReLU()(outputs)
#     outputs = Flatten()(outputs)
#     outputs = Softmax()(outputs)
    outputs = GlobalAveragePooling2D()(outputs)
    outputs = Dense(361, activation='softmax')(outputs)
    model = Model(inputs, outputs)
    
    opt = Adam(learning_rate=0.0001)
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


model = go_res()
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 19, 19, 19)  0           []                               
                                ]                                                                 
                                                                                                  
 conv5x5 (Conv2D)               (None, 19, 19, 64)   30464       ['input_2[0][0]']                
                                                                                                  
 conv1x1 (Conv2D)               (None, 19, 19, 64)   1280        ['input_2[0][0]']                
                                                                                                  
 add_18 (Add)                   (None, 19, 19, 64)   0           ['conv5x5[0][0]',          

 re_lu_48 (ReLU)                (None, 19, 19, 64)   0           ['conv2d_46[0][0]']              
                                                                                                  
 conv2d_47 (Conv2D)             (None, 19, 19, 64)   36928       ['re_lu_48[0][0]']               
                                                                                                  
 add_25 (Add)                   (None, 19, 19, 64)   0           ['re_lu_47[0][0]',               
                                                                  'conv2d_47[0][0]']              
                                                                                                  
 re_lu_49 (ReLU)                (None, 19, 19, 64)   0           ['add_25[0][0]']                 
                                                                                                  
 conv2d_48 (Conv2D)             (None, 19, 19, 64)   36928       ['re_lu_49[0][0]']               
          

                                                                                                  
 re_lu_63 (ReLU)                (None, 19, 19, 64)   0           ['add_32[0][0]']                 
                                                                                                  
 conv2d_62 (Conv2D)             (None, 19, 19, 64)   36928       ['re_lu_63[0][0]']               
                                                                                                  
 re_lu_64 (ReLU)                (None, 19, 19, 64)   0           ['conv2d_62[0][0]']              
                                                                                                  
 conv2d_63 (Conv2D)             (None, 19, 19, 64)   36928       ['re_lu_64[0][0]']               
                                                                                                  
 add_33 (Add)                   (None, 19, 19, 64)   0           ['re_lu_63[0][0]',               
          

In [26]:
# callback1 = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, 
#                                 patience=1, verbose=0, mode='min'
#                                  , restore_best_weights=True)
callback1 = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                patience=2, verbose=0, mode='max'
                                 , restore_best_weights=True)
callback2 = keras.callbacks.ModelCheckpoint('./models/model_kyu__{epoch:02d}_{val_accuracy:.3f}.h5', 
                                            monitor='val_accuracy', 
                                            verbose=0, save_best_only=False, save_weights_only=False, 
                                            mode='max', save_freq="epoch")

In [27]:
# model = load_model('./models/model_kyu_resnet_5x5p1x1_18resblo.h5')

In [28]:
history = model.fit(
    dataset,
    epochs = 500,
#     steps_per_epoch = train_steps,
    validation_data = dataset_valid,
#     validation_steps = val_steps,
    callbacks = [callback1, callback2]
)

Epoch 1/500
     16/Unknown - 31s 1s/step - loss: 5.9045 - accuracy: 9.7656e-04

KeyboardInterrupt: 

In [48]:
model.save('./models/model_kyu_resnet_fromgogame_new.h5')

In [11]:
result = model.evaluate(
    dataset_valid,
    steps = val_steps
)

105773/105773 [==============================] - 1496s 14ms/step - loss: 2.1256 - accuracy: 0.4729


In [14]:
history = model.fit(
    dataset,
    epochs = 1,
    steps_per_epoch = train_steps
)

106216/106216 [==============================] - 4941s 47ms/step - loss: 1.9110 - accuracy: 0.5071


In [15]:
model.save('./models/model_kyu_resnet_fromgogame_t050_1.h5')

In [16]:
result = model.evaluate(
    dataset_valid,
    steps = val_steps
)

105773/105773 [==============================] - 1490s 14ms/step - loss: 2.0443 - accuracy: 0.4876


In [17]:
history = model.fit(
    dataset,
    epochs = 1,
    steps_per_epoch = train_steps
)

106216/106216 [==============================] - 4265s 40ms/step - loss: 1.5184 - accuracy: 0.5739


In [18]:
model.save('./models/model_kyu_resnet_fromgogame_withnor_t050_2.h5')

In [19]:
result = model.evaluate(
    dataset_valid,
    steps = val_steps
)

105773/105773 [==============================] - 1496s 14ms/step - loss: 3.3121 - accuracy: 0.3058


In [20]:
history = model.fit(
    dataset,
    epochs = 1,
    steps_per_epoch = train_steps
)

106216/106216 [==============================] - 4263s 40ms/step - loss: 1.3174 - accuracy: 0.6168


In [21]:
model.save('./models/model_kyu_resnet_fromgogame_withnor_t050_3.h5')

In [22]:
result = model.evaluate(
    dataset_valid,
    steps = val_steps
)

105773/105773 [==============================] - 1501s 14ms/step - loss: 3.7028 - accuracy: 0.2748


In [33]:
history = model.fit(
    dataset,
    epochs = 1,
    validation_data = val_dataset
)

42/42 [==============================] - 8s 188ms/step - loss: 1.8230 - accuracy: 0.5787 - val_loss: 1.9512 - val_accuracy: 0.5478


In [ ]:
model.save('./models/model_kyu_resnet_3.h5')

In [ ]:
history = model.fit(
    dataset,
    epochs = 1
)

In [ ]:
model.save('./models/model_kyu_resnet_4.h5')

In [ ]:
history = model.fit(
    dataset,
    epochs = 1
)

In [ ]:
model.save('./models/model_kyu_10_15_f128_5.h5')

In [ ]:
history = model.fit(
    dataset,
    epochs = 1
)

In [ ]:
model.save('./models/model_kyu_10_15_f128_6.h5')

In [ ]:
history = model.fit(
    dataset,
    epochs = 1
)

In [ ]:
model.save('./models/model_kyu_10_15_f128_7.h5')

In [ ]:
history = model.fit(
    dataset,
    epochs = 1
)

In [ ]:
model.save('./models/model_kyu_10_15_f128_8.h5')

In [ ]:
history = model.fit(
    dataset,
    epochs = 1
)

In [ ]:
model.save('./models/model_kyu_10_15_f128_9.h5')

In [ ]:
history = model.fit(
    dataset,
    epochs = 1
)

In [ ]:
model.save('./models/model_kyu_10_14_11.h5')

## ALL DONE!

For using the model and creating a submission file, follow the notebook **Create Public Upload CSV.ipynb**

# End of Tutorial

You are free to use more modern NN architectures, a better pre-processing, feature extraction methods to achieve much better accuracy!